<a href="https://colab.research.google.com/github/DeepFluxion/MACK_data-visualization/blob/main/Gerador_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"""
Gerador de Datasets Realistas para Aula de Análise e Visualização de Dados
Autor: Assistant
Data: 2025
Descrição: Scripts para gerar datasets sintéticos mas realistas para ensino de
           análise de dados e conceitos de percepção visual.
"""

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import json

# Configurar seed para reprodutibilidade
np.random.seed(42)
random.seed(42)

In [ ]:
# ================================================================================
# DATASET 1: VENDAS E FATURAMENTO
# ================================================================================

def gerar_dataset_vendas():
    """
    Gera um dataset de vendas/faturamento de uma empresa fictícia
    com padrões sazonais e tendências realistas.
    """

    # Configurações
    meses = pd.date_range(start='2023-01-01', end='2024-12-31', freq='MS')
    produtos = ['Notebook Pro', 'Tablet Smart', 'Fone Wireless', 'Monitor 4K', 'Teclado Mecânico']
    regioes = ['Sul', 'Sudeste', 'Centro-Oeste', 'Nordeste', 'Norte']

    # Parâmetros base para cada produto (preço médio e volume base mensal)
    params_produtos = {
        'Notebook Pro': {'preco_medio': 3500, 'volume_base': 150, 'sazonalidade': 'alta_fim_ano'},
        'Tablet Smart': {'preco_medio': 1200, 'volume_base': 280, 'sazonalidade': 'volta_aulas'},
        'Fone Wireless': {'preco_medio': 250, 'volume_base': 850, 'sazonalidade': 'constante'},
        'Monitor 4K': {'preco_medio': 1800, 'volume_base': 220, 'sazonalidade': 'alta_meio_ano'},
        'Teclado Mecânico': {'preco_medio': 450, 'volume_base': 380, 'sazonalidade': 'gaming'}
    }

    # Peso de vendas por região (reflete população/poder de compra)
    peso_regiao = {
        'Sul': 0.20,
        'Sudeste': 0.40,
        'Centro-Oeste': 0.15,
        'Nordeste': 0.18,
        'Norte': 0.07
    }

    # Função para aplicar sazonalidade
    def aplicar_sazonalidade(mes, tipo_sazonalidade):
        mes_num = mes.month

        if tipo_sazonalidade == 'alta_fim_ano':
            # Black Friday e Natal
            if mes_num in [11, 12]:
                return 1.4 + np.random.uniform(-0.1, 0.1)
            elif mes_num in [1, 2]:
                return 0.8 + np.random.uniform(-0.05, 0.05)
            else:
                return 1.0 + np.random.uniform(-0.1, 0.1)

        elif tipo_sazonalidade == 'volta_aulas':
            # Janeiro/Fevereiro e Julho/Agosto
            if mes_num in [1, 2, 7, 8]:
                return 1.3 + np.random.uniform(-0.1, 0.1)
            else:
                return 0.95 + np.random.uniform(-0.05, 0.05)

        elif tipo_sazonalidade == 'gaming':
            # Férias e fim de ano
            if mes_num in [6, 7, 11, 12]:
                return 1.25 + np.random.uniform(-0.1, 0.1)
            else:
                return 0.95 + np.random.uniform(-0.05, 0.05)

        elif tipo_sazonalidade == 'alta_meio_ano':
            # Home office e meio do ano
            if mes_num in [5, 6, 7]:
                return 1.2 + np.random.uniform(-0.1, 0.1)
            else:
                return 1.0 + np.random.uniform(-0.05, 0.05)
        else:
            return 1.0 + np.random.uniform(-0.05, 0.05)

    # Gerar dados
    dados = []

    for mes in meses:
        for produto in produtos:
            for regiao in regioes:
                # Parâmetros do produto
                params = params_produtos[produto]

                # Volume base com variação regional
                volume = params['volume_base'] * peso_regiao[regiao]

                # Aplicar sazonalidade
                fator_sazonal = aplicar_sazonalidade(mes, params['sazonalidade'])
                volume = int(volume * fator_sazonal)

                # Adicionar tendência de crescimento (empresa em expansão)
                meses_desde_inicio = (mes.year - 2023) * 12 + mes.month - 1
                fator_crescimento = 1 + (meses_desde_inicio * 0.008)  # 0.8% ao mês
                volume = int(volume * fator_crescimento)

                # Adicionar ruído aleatório
                volume = max(0, int(volume + np.random.normal(0, volume * 0.1)))

                # Calcular preço com pequena variação
                preco_unitario = params['preco_medio'] * (1 + np.random.uniform(-0.05, 0.05))

                # Calcular faturamento
                faturamento = volume * preco_unitario

                # Adicionar evento especial (promoção) ocasionalmente
                if np.random.random() < 0.05:  # 5% de chance
                    preco_unitario *= 0.85  # 15% de desconto
                    volume = int(volume * 1.3)  # 30% mais vendas
                    faturamento = volume * preco_unitario

                dados.append({
                    'data': mes,
                    'ano': mes.year,
                    'mes': mes.month,
                    'mes_nome': mes.strftime('%B'),
                    'trimestre': f'Q{(mes.month-1)//3 + 1}',
                    'produto': produto,
                    'categoria': 'Eletrônicos' if produto in ['Notebook Pro', 'Tablet Smart', 'Monitor 4K'] else 'Acessórios',
                    'regiao': regiao,
                    'quantidade_vendida': volume,
                    'preco_unitario': round(preco_unitario, 2),
                    'faturamento': round(faturamento, 2),
                    'custo_unitario': round(preco_unitario * 0.6, 2),  # Margem de 40%
                    'lucro_bruto': round(faturamento * 0.4, 2)
                })

    df_vendas = pd.DataFrame(dados)

    # Adicionar algumas métricas calculadas úteis
    df_vendas['margem_lucro'] = round((df_vendas['lucro_bruto'] / df_vendas['faturamento']) * 100, 2)
    df_vendas['ticket_medio'] = round(df_vendas['faturamento'] / df_vendas['quantidade_vendida'], 2)

    return df_vendas

In [ ]:
# ================================================================================
# DATASET 2: ATENDIMENTO AO CLIENTE / SUPORTE
# ================================================================================

def gerar_dataset_atendimento():
    """
    Gera um dataset de atendimentos/tickets de suporte com múltiplos canais
    e métricas de desempenho.
    """

    # Configurações
    meses = pd.date_range(start='2023-01-01', end='2024-12-31', freq='MS')
    canais = ['Telefone', 'E-mail', 'Chat Online', 'WhatsApp', 'Presencial']
    tipos_problema = ['Técnico', 'Financeiro', 'Informações', 'Reclamação', 'Sugestão']
    prioridades = ['Baixa', 'Média', 'Alta', 'Crítica']

    # Distribuição base de tickets por canal
    dist_canal = {
        'Telefone': 0.25,
        'E-mail': 0.30,
        'Chat Online': 0.20,
        'WhatsApp': 0.18,
        'Presencial': 0.07
    }

    # Tempo médio de resolução por canal (em horas)
    tempo_resolucao_base = {
        'Telefone': 2.5,
        'E-mail': 24,
        'Chat Online': 1.5,
        'WhatsApp': 4,
        'Presencial': 1
    }

    # Satisfação média por canal (1-5)
    satisfacao_base = {
        'Telefone': 3.8,
        'E-mail': 3.5,
        'Chat Online': 4.2,
        'WhatsApp': 4.0,
        'Presencial': 4.5
    }

    dados = []

    for mes in meses:
        # Volume base de tickets (com crescimento da empresa)
        meses_desde_inicio = (mes.year - 2023) * 12 + mes.month - 1
        volume_base_mes = 1000 + (meses_desde_inicio * 30)  # Crescimento de 30 tickets/mês

        # Sazonalidade (mais tickets em determinados períodos)
        if mes.month in [1, 7]:  # Janeiro e Julho
            volume_base_mes *= 1.3
        elif mes.month in [11, 12]:  # Black Friday e Natal
            volume_base_mes *= 1.5

        for canal in canais:
            # Volume de tickets para este canal
            volume_canal = int(volume_base_mes * dist_canal[canal] * np.random.uniform(0.9, 1.1))

            # Distribuir por tipo de problema
            for tipo in tipos_problema:
                # Volume por tipo (distribuição não uniforme)
                if tipo == 'Técnico':
                    peso_tipo = 0.35
                elif tipo == 'Financeiro':
                    peso_tipo = 0.20
                elif tipo == 'Informações':
                    peso_tipo = 0.25
                elif tipo == 'Reclamação':
                    peso_tipo = 0.15
                else:  # Sugestão
                    peso_tipo = 0.05

                volume_tipo = int(volume_canal * peso_tipo)

                # Distribuir por prioridade
                for prioridade in prioridades:
                    if prioridade == 'Baixa':
                        peso_prio = 0.40
                    elif prioridade == 'Média':
                        peso_prio = 0.35
                    elif prioridade == 'Alta':
                        peso_prio = 0.20
                    else:  # Crítica
                        peso_prio = 0.05

                    volume_final = max(1, int(volume_tipo * peso_prio * np.random.uniform(0.8, 1.2)))

                    # Calcular métricas
                    tempo_base = tempo_resolucao_base[canal]

                    # Ajustar tempo por prioridade
                    if prioridade == 'Crítica':
                        tempo_base *= 0.5
                    elif prioridade == 'Alta':
                        tempo_base *= 0.7
                    elif prioridade == 'Baixa':
                        tempo_base *= 1.5

                    # Adicionar variação
                    tempo_resolucao = max(0.5, tempo_base + np.random.normal(0, tempo_base * 0.3))

                    # Satisfação (inversamente relacionada ao tempo de resolução)
                    satisfacao = satisfacao_base[canal]
                    if tempo_resolucao > tempo_base * 1.5:
                        satisfacao -= 0.5
                    elif tempo_resolucao < tempo_base * 0.8:
                        satisfacao += 0.3
                    satisfacao = max(1, min(5, satisfacao + np.random.normal(0, 0.3)))

                    # Taxa de resolução no primeiro contato
                    if canal in ['Telefone', 'Presencial', 'Chat Online']:
                        taxa_primeiro_contato = 0.7 + np.random.uniform(-0.1, 0.1)
                    else:
                        taxa_primeiro_contato = 0.4 + np.random.uniform(-0.1, 0.1)

                    dados.append({
                        'data': mes,
                        'ano': mes.year,
                        'mes': mes.month,
                        'mes_nome': mes.strftime('%B'),
                        'trimestre': f'Q{(mes.month-1)//3 + 1}',
                        'canal': canal,
                        'tipo_problema': tipo,
                        'prioridade': prioridade,
                        'quantidade_tickets': volume_final,
                        'tempo_medio_resolucao_horas': round(tempo_resolucao, 2),
                        'satisfacao_media': round(satisfacao, 2),
                        'taxa_resolucao_primeiro_contato': round(taxa_primeiro_contato, 2),
                        'tickets_reabertos': max(0, int(volume_final * (1 - taxa_primeiro_contato) * 0.3)),
                        'custo_medio_por_ticket': round(tempo_resolucao * 25, 2)  # R$25/hora
                    })

    df_atendimento = pd.DataFrame(dados)

    # Adicionar métricas calculadas
    df_atendimento['eficiencia_canal'] = round(
        (df_atendimento['satisfacao_media'] / 5) *
        df_atendimento['taxa_resolucao_primeiro_contato'] * 100, 2
    )

    return df_atendimento

In [ ]:
# ================================================================================
# DATASET 3: PESQUISA DE MERCADO / HÁBITOS DO CONSUMIDOR
# ================================================================================

def gerar_dataset_pesquisa_mercado():
    """
    Gera um dataset de pesquisa de mercado sobre uso de serviços e produtos.
    """

    # Configurações para pesquisa trimestral
    trimestres = pd.date_range(start='2023-01-01', end='2024-12-31', freq='QS')

    # Serviços pesquisados
    servicos = [
        'Internet Banda Larga',
        'Streaming de Vídeo',
        'Streaming de Música',
        'Delivery de Comida',
        'E-commerce',
        'Serviços Bancários Digitais',
        'Redes Sociais',
        'Aplicativos de Transporte'
    ]

    # Faixas etárias
    faixas_etarias = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']

    # Regiões
    regioes = ['Sul', 'Sudeste', 'Centro-Oeste', 'Nordeste', 'Norte']

    # Classes sociais
    classes = ['A', 'B', 'C', 'D/E']

    # Base de penetração por serviço (% da população)
    penetracao_base = {
        'Internet Banda Larga': 0.75,
        'Streaming de Vídeo': 0.65,
        'Streaming de Música': 0.45,
        'Delivery de Comida': 0.40,
        'E-commerce': 0.70,
        'Serviços Bancários Digitais': 0.60,
        'Redes Sociais': 0.80,
        'Aplicativos de Transporte': 0.35
    }

    # Tendência de crescimento anual
    crescimento_anual = {
        'Internet Banda Larga': 0.05,
        'Streaming de Vídeo': 0.15,
        'Streaming de Música': 0.12,
        'Delivery de Comida': 0.10,
        'E-commerce': 0.08,
        'Serviços Bancários Digitais': 0.20,
        'Redes Sociais': 0.03,
        'Aplicativos de Transporte': 0.08
    }

    dados = []

    for trimestre in trimestres:
        trimestre_num = (trimestre.month - 1) // 3 + 1
        ano = trimestre.year

        # Calcular quantos trimestres se passaram desde o início
        trimestres_desde_inicio = (ano - 2023) * 4 + trimestre_num - 1

        for servico in servicos:
            # Aplicar crescimento
            penetracao_atual = penetracao_base[servico] * (1 + crescimento_anual[servico] * (trimestres_desde_inicio / 4))
            penetracao_atual = min(0.95, penetracao_atual)  # Máximo 95%

            for faixa in faixas_etarias:
                for regiao in regioes:
                    for classe in classes:
                        # Ajustar penetração por demografia
                        ajuste = 1.0

                        # Ajuste por idade
                        if servico in ['Streaming de Vídeo', 'Streaming de Música', 'Redes Sociais']:
                            if faixa in ['18-24', '25-34']:
                                ajuste *= 1.3
                            elif faixa in ['55-64', '65+']:
                                ajuste *= 0.6
                        elif servico == 'Serviços Bancários Digitais':
                            if faixa in ['25-34', '35-44']:
                                ajuste *= 1.2
                            elif faixa == '65+':
                                ajuste *= 0.5

                        # Ajuste por região
                        if regiao in ['Sul', 'Sudeste']:
                            ajuste *= 1.1
                        elif regiao in ['Norte', 'Nordeste']:
                            ajuste *= 0.85

                        # Ajuste por classe social
                        if classe == 'A':
                            ajuste *= 1.4
                        elif classe == 'B':
                            ajuste *= 1.2
                        elif classe == 'C':
                            ajuste *= 0.9
                        else:  # D/E
                            ajuste *= 0.5

                        # Calcular penetração final
                        penetracao_final = min(1.0, penetracao_atual * ajuste)

                        # Tamanho da amostra (simulado)
                        tamanho_amostra = np.random.randint(800, 1200)

                        # Número de usuários
                        usuarios = int(tamanho_amostra * penetracao_final)

                        # Frequência de uso (vezes por semana)
                        if servico in ['Redes Sociais', 'Streaming de Vídeo']:
                            freq_media = 6.5 * ajuste
                        elif servico in ['E-commerce', 'Delivery de Comida']:
                            freq_media = 2.5 * ajuste
                        else:
                            freq_media = 4.0 * ajuste

                        frequencia_uso = max(0.5, freq_media + np.random.normal(0, 1))

                        # Satisfação (1-10)
                        satisfacao = min(10, max(1, 7 + np.random.normal(0, 1.5)))

                        # Intenção de continuar usando (%)
                        intencao_continuar = min(100, max(0,
                            penetracao_final * 100 + np.random.normal(0, 10)))

                        dados.append({
                            'data': trimestre,
                            'ano': ano,
                            'trimestre': f'Q{trimestre_num}',
                            'servico': servico,
                            'faixa_etaria': faixa,
                            'regiao': regiao,
                            'classe_social': classe,
                            'tamanho_amostra': tamanho_amostra,
                            'usuarios_ativos': usuarios,
                            'penetracao_percentual': round(penetracao_final * 100, 2),
                            'frequencia_uso_semanal': round(frequencia_uso, 2),
                            'satisfacao_media': round(satisfacao, 2),
                            'intencao_continuar_pct': round(intencao_continuar, 2),
                            'nao_usuarios': tamanho_amostra - usuarios,
                            'potencial_crescimento_pct': round((1 - penetracao_final) * 100, 2)
                        })

    df_pesquisa = pd.DataFrame(dados)

    # Adicionar NPS (Net Promoter Score) calculado
    df_pesquisa['nps_estimado'] = df_pesquisa['satisfacao_media'].apply(
        lambda x: round((x - 5) * 20 if x > 7 else (x - 5) * 10 if x > 5 else (x - 5) * 5, 1)
    )

    return df_pesquisa

In [ ]:
# ================================================================================
# DATASET 4: DADOS AGREGADOS SIMPLIFICADOS (Para exercícios iniciais)
# ================================================================================

def gerar_dataset_simplificado():
    """
    Gera datasets simplificados e agregados para exercícios iniciais.
    """

    # 1. Vendas mensais simplificadas (apenas totais)
    meses = pd.date_range(start='2024-01-01', end='2024-12-31', freq='MS')
    vendas_simples = []

    base_vendas = 500000
    for i, mes in enumerate(meses):
        # Padrão sazonal simples
        if mes.month in [11, 12]:  # Black Friday e Natal
            fator = 1.4
        elif mes.month in [1, 2]:  # Início do ano
            fator = 0.8
        elif mes.month in [6, 7]:  # Meio do ano
            fator = 1.1
        else:
            fator = 1.0

        # Tendência de crescimento
        crescimento = 1 + (i * 0.02)

        valor = base_vendas * fator * crescimento * np.random.uniform(0.95, 1.05)

        vendas_simples.append({
            'mes': mes.strftime('%Y-%m'),
            'mes_nome': mes.strftime('%B'),
            'faturamento_total': round(valor, 2),
            'numero_vendas': int(valor / 850),
            'ticket_medio': round(valor / int(valor / 850), 2)
        })

    df_vendas_simples = pd.DataFrame(vendas_simples)

    # 2. Comparação de produtos (snapshot)
    produtos_comparacao = pd.DataFrame({
        'produto': ['Notebook Pro', 'Tablet Smart', 'Fone Wireless', 'Monitor 4K', 'Teclado Mecânico'],
        'vendas_2023': [1850, 3400, 10200, 2640, 4560],
        'vendas_2024': [2200, 3850, 11500, 3100, 5200],
        'preco_medio': [3500, 1200, 250, 1800, 450],
        'margem_lucro_pct': [35, 40, 45, 38, 42],
        'satisfacao_cliente': [4.2, 4.5, 4.1, 4.3, 4.6]
    })

    produtos_comparacao['crescimento_pct'] = round(
        ((produtos_comparacao['vendas_2024'] - produtos_comparacao['vendas_2023']) /
         produtos_comparacao['vendas_2023']) * 100, 2
    )

    # 3. Distribuição por categoria (para gráfico de pizza)
    categorias = pd.DataFrame({
        'categoria': ['Eletrônicos', 'Acessórios', 'Software', 'Serviços', 'Outros'],
        'faturamento_anual': [8500000, 3200000, 1800000, 2100000, 650000],
        'numero_clientes': [15000, 22000, 8500, 5200, 3100]
    })

    categorias['percentual_faturamento'] = round(
        (categorias['faturamento_anual'] / categorias['faturamento_anual'].sum()) * 100, 2
    )

    # 4. Satisfação por canal de atendimento
    canais_satisfacao = pd.DataFrame({
        'canal': ['Telefone', 'E-mail', 'Chat Online', 'WhatsApp', 'Presencial'],
        'volume_mensal': [2500, 3000, 2000, 1800, 700],
        'satisfacao_media': [3.8, 3.5, 4.2, 4.0, 4.5],
        'tempo_resposta_horas': [2.5, 24, 1.5, 4, 1],
        'custo_por_atendimento': [15, 8, 5, 6, 25]
    })

    return {
        'vendas_mensais': df_vendas_simples,
        'produtos_comparacao': produtos_comparacao,
        'categorias': categorias,
        'canais_satisfacao': canais_satisfacao
    }

# ================================================================================
# FUNÇÃO PRINCIPAL PARA GERAR TODOS OS DATASETS
# ================================================================================

def gerar_todos_datasets():
    """
    Gera todos os datasets e os salva em arquivos CSV.
    """

    print("Gerando datasets para a aula de visualização de dados...")
    print("-" * 60)

    # 1. Dataset de Vendas
    print("1. Gerando dataset de vendas e faturamento...")
    df_vendas = gerar_dataset_vendas()
    df_vendas.to_csv('dataset_vendas_completo.csv', index=False, encoding='utf-8')
    print(f"   ✓ Criado: dataset_vendas_completo.csv")
    print(f"   - {len(df_vendas)} registros")
    print(f"   - Período: {df_vendas['data'].min().strftime('%Y-%m')} a {df_vendas['data'].max().strftime('%Y-%m')}")
    print(f"   - Produtos: {df_vendas['produto'].nunique()}")
    print(f"   - Regiões: {df_vendas['regiao'].nunique()}")
    print()

    # 2. Dataset de Atendimento
    print("2. Gerando dataset de atendimento ao cliente...")
    df_atendimento = gerar_dataset_atendimento()
    df_atendimento.to_csv('dataset_atendimento_completo.csv', index=False, encoding='utf-8')
    print(f"   ✓ Criado: dataset_atendimento_completo.csv")
    print(f"   - {len(df_atendimento)} registros")
    print(f"   - Canais: {df_atendimento['canal'].nunique()}")
    print(f"   - Tipos de problema: {df_atendimento['tipo_problema'].nunique()}")
    print()

    # 3. Dataset de Pesquisa de Mercado
    print("3. Gerando dataset de pesquisa de mercado...")
    df_pesquisa = gerar_dataset_pesquisa_mercado()
    df_pesquisa.to_csv('dataset_pesquisa_mercado.csv', index=False, encoding='utf-8')
    print(f"   ✓ Criado: dataset_pesquisa_mercado.csv")
    print(f"   - {len(df_pesquisa)} registros")
    print(f"   - Serviços pesquisados: {df_pesquisa['servico'].nunique()}")
    print(f"   - Segmentos demográficos: {df_pesquisa['faixa_etaria'].nunique() * df_pesquisa['classe_social'].nunique()}")
    print()

    # 4. Datasets Simplificados
    print("4. Gerando datasets simplificados para exercícios...")
    datasets_simples = gerar_dataset_simplificado()

    for nome, df in datasets_simples.items():
        filename = f'dataset_{nome}.csv'
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"   ✓ Criado: {filename} ({len(df)} registros)")

    print()
    print("-" * 60)
    print("✅ Todos os datasets foram gerados com sucesso!")

    # Retornar resumo estatístico
    return {
        'vendas': df_vendas,
        'atendimento': df_atendimento,
        'pesquisa': df_pesquisa,
        'simplificados': datasets_simples
    }

# ================================================================================
# FUNÇÃO PARA GERAR DESCRIÇÃO DOS DATASETS
# ================================================================================

def gerar_documentacao_datasets():
    """
    Gera documentação detalhada sobre os datasets criados.
    """

    doc = """
# DOCUMENTAÇÃO DOS DATASETS GERADOS
=====================================

## 1. DATASET DE VENDAS E FATURAMENTO (dataset_vendas_completo.csv)

### Descrição:
Dataset com 24 meses de dados de vendas de uma empresa de tecnologia,
incluindo 5 produtos em 5 regiões do Brasil.

### Colunas:
- data: Data (primeiro dia do mês)
- ano: Ano
- mes: Número do mês
- mes_nome: Nome do mês
- trimestre: Trimestre (Q1-Q4)
- produto: Nome do produto
- categoria: Categoria do produto (Eletrônicos/Acessórios)
- regiao: Região do Brasil
- quantidade_vendida: Unidades vendidas
- preco_unitario: Preço unitário em R$
- faturamento: Faturamento total em R$
- custo_unitario: Custo unitário em R$
- lucro_bruto: Lucro bruto em R$
- margem_lucro: Margem de lucro em %
- ticket_medio: Ticket médio em R$

### Características dos dados:
- Sazonalidade: Black Friday/Natal (alta), Janeiro/Fevereiro (baixa)
- Tendência: Crescimento médio de 0.8% ao mês
- Produtos com diferentes padrões sazonais
- Distribuição regional baseada em população/poder de compra

### Perguntas sugeridas para análise:
1. Qual foi o faturamento total por trimestre?
2. Qual produto teve maior crescimento percentual?
3. Como as vendas variam por região?
4. Qual o impacto da sazonalidade nas vendas?
5. Qual categoria é mais lucrativa?

---

## 2. DATASET DE ATENDIMENTO AO CLIENTE (dataset_atendimento_completo.csv)

### Descrição:
Dataset com 24 meses de dados de atendimento ao cliente através de
múltiplos canais, incluindo métricas de desempenho e satisfação.

### Colunas:
- data: Data (primeiro dia do mês)
- ano: Ano
- mes: Número do mês
- mes_nome: Nome do mês
- trimestre: Trimestre
- canal: Canal de atendimento
- tipo_problema: Classificação do problema
- prioridade: Nível de prioridade
- quantidade_tickets: Volume de tickets
- tempo_medio_resolucao_horas: Tempo médio de resolução
- satisfacao_media: Nota de satisfação (1-5)
- taxa_resolucao_primeiro_contato: Taxa de FCR
- tickets_reabertos: Quantidade de reaberturas
- custo_medio_por_ticket: Custo médio em R$
- eficiencia_canal: Índice de eficiência (0-100)

### Características dos dados:
- Crescimento no volume: 30 tickets/mês
- Picos em Janeiro, Julho, Novembro e Dezembro
- Canais com diferentes perfis de eficiência
- Correlação entre tempo de resolução e satisfação

### Perguntas sugeridas para análise:
1. Qual canal é mais eficiente?
2. Como o volume de tickets evolui ao longo do tempo?
3. Existe correlação entre prioridade e tempo de resolução?
4. Qual tipo de problema é mais frequente?
5. Como reduzir o custo total de atendimento?

---

## 3. DATASET DE PESQUISA DE MERCADO (dataset_pesquisa_mercado.csv)

### Descrição:
Dataset trimestral com dados de penetração e uso de serviços digitais,
segmentado por demografia (idade, região, classe social).

### Colunas:
- data: Data (primeiro dia do trimestre)
- ano: Ano
- trimestre: Trimestre
- servico: Tipo de serviço digital
- faixa_etaria: Faixa de idade
- regiao: Região do Brasil
- classe_social: Classe socioeconômica
- tamanho_amostra: Tamanho da amostra pesquisada
- usuarios_ativos: Número de usuários ativos
- penetracao_percentual: Taxa de penetração em %
- frequencia_uso_semanal: Frequência de uso (vezes/semana)
- satisfacao_media: Satisfação (1-10)
- intencao_continuar_pct: Intenção de continuar usando em %
- nao_usuarios: Número de não usuários
- potencial_crescimento_pct: Potencial de crescimento em %
- nps_estimado: Net Promoter Score estimado

### Características dos dados:
- 8 serviços digitais diferentes
- 6 faixas etárias
- 5 regiões
- 4 classes sociais
- Tendências de crescimento variadas por serviço

### Perguntas sugeridas para análise:
1. Qual serviço tem maior penetração?
2. Como a adoção varia por faixa etária?
3. Quais regiões têm maior potencial de crescimento?
4. Existe correlação entre classe social e uso de serviços?
5. Quais serviços estão crescendo mais rápido?

---

## 4. DATASETS SIMPLIFICADOS (Para exercícios iniciais)

### 4.1 Vendas Mensais (dataset_vendas_mensais.csv)
- Dados agregados mensais de faturamento
- Ideal para gráficos de linha simples
- 12 meses de dados

### 4.2 Comparação de Produtos (dataset_produtos_comparacao.csv)
- Snapshot comparativo de 5 produtos
- Ideal para gráficos de barras
- Inclui crescimento ano a ano

### 4.3 Categorias (dataset_categorias.csv)
- Distribuição de faturamento por categoria
- Ideal para gráficos de pizza/donut
- Inclui percentuais calculados

### 4.4 Canais de Satisfação (dataset_canais_satisfacao.csv)
- Métricas de desempenho por canal
- Ideal para análises de correlação
- Combina volume, satisfação e custo

---

## EXEMPLOS DE USO DOS DATASETS

### Exemplo 1: Análise de Tendência Temporal
```python
import pandas as pd
import matplotlib.pyplot as plt

# Carregar dados
df = pd.read_csv('dataset_vendas_completo.csv')
df['data'] = pd.to_datetime(df['data'])

# Agregar por mês
vendas_mensais = df.groupby('data')['faturamento'].sum()

# Plotar
plt.figure(figsize=(12, 6))
plt.plot(vendas_mensais.index, vendas_mensais.values)
plt.title('Evolução do Faturamento Mensal')
plt.xlabel('Mês')
plt.ylabel('Faturamento (R$)')
plt.grid(True, alpha=0.3)
plt.show()
```

### Exemplo 2: Comparação entre Categorias
```python
# Carregar dados simplificados
df_cat = pd.read_csv('dataset_categorias.csv')

# Criar gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(df_cat['categoria'], df_cat['faturamento_anual'])
plt.title('Faturamento por Categoria')
plt.xlabel('Categoria')
plt.ylabel('Faturamento Anual (R$)')
plt.xticks(rotation=45)
plt.show()
```

### Exemplo 3: Análise de Correlação
```python
# Carregar dados de atendimento
df_at = pd.read_csv('dataset_atendimento_completo.csv')

# Calcular correlação
correlacao = df_at[['tempo_medio_resolucao_horas',
                     'satisfacao_media']].corr()

# Visualizar
plt.figure(figsize=(8, 6))
plt.scatter(df_at['tempo_medio_resolucao_horas'],
           df_at['satisfacao_media'], alpha=0.5)
plt.xlabel('Tempo de Resolução (horas)')
plt.ylabel('Satisfação Média')
plt.title('Relação entre Tempo de Resolução e Satisfação')
plt.show()
```

---

## NOTAS IMPORTANTES

1. **Reprodutibilidade**: Os datasets usam seed fixo (42) para permitir reprodução
2. **Realismo**: Incluem variações, sazonalidades e ruído similares a dados reais
3. **Complexidade Gradual**: Desde datasets simples até complexos
4. **Flexibilidade**: Podem ser filtrados/agregados para diferentes exercícios
5. **Encoding**: Todos salvos em UTF-8 para compatibilidade

"""

    return doc

# ================================================================================
# EXECUTAR GERAÇÃO DE DATASETS
# ================================================================================

if __name__ == "__main__":
    # Gerar todos os datasets
    datasets = gerar_todos_datasets()

    # Gerar documentação
    doc = gerar_documentacao_datasets()

    # Salvar documentação
    with open('DOCUMENTACAO_DATASETS.md', 'w', encoding='utf-8') as f:
        f.write(doc)

    print("\n📄 Documentação salva em: DOCUMENTACAO_DATASETS.md")

    # Mostrar estatísticas resumidas
    print("\n" + "=" * 60)
    print("ESTATÍSTICAS RESUMIDAS DOS DATASETS")
    print("=" * 60)

    # Dataset de Vendas
    df_vendas = datasets['vendas']
    print("\n📊 Dataset de Vendas:")
    print(f"   - Faturamento Total: R$ {df_vendas['faturamento'].sum():,.2f}")
    print(f"   - Ticket Médio: R$ {df_vendas['ticket_medio'].mean():.2f}")
    print(f"   - Margem Média: {df_vendas['margem_lucro'].mean():.1f}%")

    # Dataset de Atendimento
    df_atend = datasets['atendimento']
    print("\n📞 Dataset de Atendimento:")
    print(f"   - Total de Tickets: {df_atend['quantidade_tickets'].sum():,}")
    print(f"   - Satisfação Média: {df_atend['satisfacao_media'].mean():.2f}/5.0")
    print(f"   - Tempo Médio de Resolução: {df_atend['tempo_medio_resolucao_horas'].mean():.1f}h")

    # Dataset de Pesquisa
    df_pesq = datasets['pesquisa']
    print("\n🔍 Dataset de Pesquisa de Mercado:")
    print(f"   - Total de Respondentes: {df_pesq['tamanho_amostra'].sum():,}")
    print(f"   - Penetração Média dos Serviços: {df_pesq['penetracao_percentual'].mean():.1f}%")
    print(f"   - NPS Médio: {df_pesq['nps_estimado'].mean():.1f}")

    print("\n" + "=" * 60)
    print("✅ Processo concluído com sucesso!")
    print("📁 Arquivos CSV gerados no diretório atual")
    print("📚 Prontos para uso na aula de visualização de dados")
    print("=" * 60)

Gerando datasets para a aula de visualização de dados...
------------------------------------------------------------
1. Gerando dataset de vendas e faturamento...
   ✓ Criado: dataset_vendas_completo.csv
   - 600 registros
   - Período: 2023-01 a 2024-12
   - Produtos: 5
   - Regiões: 5

2. Gerando dataset de atendimento ao cliente...
   ✓ Criado: dataset_atendimento_completo.csv
   - 2400 registros
   - Canais: 5
   - Tipos de problema: 5

3. Gerando dataset de pesquisa de mercado...
   ✓ Criado: dataset_pesquisa_mercado.csv
   - 7680 registros
   - Serviços pesquisados: 8
   - Segmentos demográficos: 24

4. Gerando datasets simplificados para exercícios...
   ✓ Criado: dataset_vendas_mensais.csv (12 registros)
   ✓ Criado: dataset_produtos_comparacao.csv (5 registros)
   ✓ Criado: dataset_categorias.csv (5 registros)
   ✓ Criado: dataset_canais_satisfacao.csv (5 registros)

------------------------------------------------------------
✅ Todos os datasets foram gerados com sucesso!

📄